https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/image_classification.ipynb#scrollTo=7tjOWPQYLq4u

gotta get those dependencies

In [72]:
from transformers import ImageGPTFeatureExtractor, ImageGPTModel
import torch
from torchvision.datasets import ImageFolder
from torchvision import datasets, transforms
import torch.nn as nn
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.optim import Adam

alright, so I found a model that can give us the pretrained benefits while not loosing information from scaling up the image

In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_checkpoint = 'openai/imagegpt-small'
feature_extractor = ImageGPTFeatureExtractor.from_pretrained("openai/imagegpt-small")
ImageGPT = ImageGPTModel.from_pretrained("openai/imagegpt-small")
ImageGPT.to(device)

c:\Users\prime\anaconda3\envs\DL\lib\site-packages\transformers\models\imagegpt\feature_extraction_imagegpt.py:28: FutureWarning: The class ImageGPTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ImageGPTImageProcessor instead.
  warnings.warn(
Some weights of the model checkpoint at openai/imagegpt-small were not used when initializing ImageGPTModel: ['lm_head.weight']
- This IS expected if you are initializing ImageGPTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ImageGPTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ImageGPTModel(
  (wte): Embedding(513, 512)
  (wpe): Embedding(1024, 512)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-23): 24 x ImageGPTBlock(
      (ln_1): ImageGPTLayerNorm()
      (attn): ImageGPTAttention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): ImageGPTLayerNorm()
      (mlp): ImageGPTMLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): QuickGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): ImageGPTLayerNorm()
)

this version of the model just outputs hidden states so we need to create a classification head

In [93]:
num_classes = 3  
class ClassificationHead(nn.Module):
    def __init__(self, input_size, num_classes):
        super(ClassificationHead, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)   # Fully connected layer 1
        self.fc2 = nn.Linear(128, num_classes)  # Fully connected layer 2 (output layer)
        self.softmax = nn.Softmax(dim=1)        # Softmax activation for multi-class classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))    # Apply ReLU activation to the first fully connected layer
        x = self.fc2(x)                # Get the logits (scores) for each class
        output = self.softmax(x)       # Apply softmax to get class probabilities
        return output

this combines ImageGPT with new classification head

In [127]:
class VIT(nn.Module):
    def __init__(self, num_classes):
        super(VIT, self).__init__()
        # Load or initialize the ImageGPT model
        self.image_gpt = ImageGPTModel.from_pretrained("openai/imagegpt-small")
        
        # Define the classification head
        self.classification_head = ClassificationHead(self.image_gpt.config.hidden_size, num_classes)

    def forward(self, images):
        # images = torch.tensor(images).to(torch.int64)
        print(f'imgae type: {type(images)}')
        # images = torch.tensor(images).to(device).long()
        # Apparently this adresses the bug 

        images = images.type(torch.LongTensor)
        b_input_mask = b_input_mask.type(torch.LongTensor)
        b_labels = b_labels.type(torch.LongTensor)

        b_input_ids = b_input_ids.to(device)
        b_input_mask = b_input_mask.to(device)
        b_labels = b_labels.to(device)


        # Pass the images through the ImageGPT model
        features = self.image_gpt(images)

        # Pass the pooled output through the classification head
        output = self.classification_head(features)
        
        return output


here I need to load the data in a compatible way 

In [128]:
data_dir = "../data/wine_data/images-standard/"

In [129]:
from datasets import load_dataset 

# load a custom dataset from local/remote files or folders using the ImageFolder feature

# option 1: local/remote files (supporting the following formats: tar, gzip, zip, xz, rar, zstd)
dataset = load_dataset("imagefolder", data_dir=data_dir)
dataset


Resolving data files: 100%|██████████| 170/170 [00:00<?, ?it/s]


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 170
    })
})

id2string dictionary to decode 

In [130]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'3'

## preprocessing

In [131]:
from transformers import AutoImageProcessor

image_processor  = AutoImageProcessor.from_pretrained('openai/imagegpt-small')
image_processor 

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


ImageGPTImageProcessor {
  "clusters": [
    [
      0.8866443634033203,
      0.6618829369544983,
      0.3891746401786804
    ],
    [
      -0.6042559146881104,
      -0.02295008860528469,
      0.5423797369003296
    ],
    [
      0.12942790985107422,
      0.03570118546485901,
      -0.3643915355205536
    ],
    [
      -0.3553103804588318,
      -0.15857496857643127,
      -0.664303183555603
    ],
    [
      0.7844981551170349,
      -0.2559399902820587,
      -0.7189618945121765
    ],
    [
      0.1935412883758545,
      0.23239648342132568,
      0.08234082162380219
    ],
    [
      -0.9012048840522766,
      -0.7926875352859497,
      -0.9216221570968628
    ],
    [
      -0.013770446181297302,
      -0.8677454590797424,
      -0.8910260796546936
    ],
    [
      -0.8732763528823853,
      -0.5560516715049744,
      -0.686274528503418
    ],
    [
      -0.3679597079753876,
      -0.4616401493549347,
      -0.8443512916564941
    ],
    [
      -0.17434515058994293,

In [178]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
        [
            Resize(size),
            ToTensor(),
        ]
    )

val_transforms = Compose(
        [
            Resize(size),
            # CenterCrop(crop_size),
            ToTensor(),
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in example_batch["image"]]
    # x = torch.tensor(example_batch['pixel_values']).to(torch.int64)
    # example_batch["pixel_values"] = torch.tensor(example_batch['pixel_values']).to(device).long()
    example_batch['pixel_values'] = [tensor.long() for tensor in example_batch['pixel_values']]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    # x = torch.tensor(example_batch['pixel_values']).to(torch.int64)
    # example_batch["pixel_values"] = torch.tensor(example_batch['pixel_values']).to(device).long()
    example_batch['pixel_values'] = [tensor.long() for tensor in example_batch['pixel_values']]
    return example_batch

In [179]:
# split up training into training + validation
splits = dataset["train"].train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

In [180]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [181]:
# type(train_ds[0]['label'])
train_ds[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=64x64>,
 'label': 1,
 'pixel_values': tensor([[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]])}

In [182]:
train_ds[0]['pixel_values'].shape

torch.Size([3, 32, 32])

In [183]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    'openai/imagegpt-small', 
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

Some weights of the model checkpoint at openai/imagegpt-small were not used when initializing ImageGPTForImageClassification: ['lm_head.weight']
- This IS expected if you are initializing ImageGPTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ImageGPTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ImageGPTForImageClassification were not initialized from the model checkpoint at openai/imagegpt-small and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [184]:
batch_size = 32 # batch size for training and evaluation

model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

In [185]:
import numpy as np

# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [186]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [187]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [188]:
train_results = trainer.train()

  0%|          | 0/3 [03:17<?, ?it/s]
c:\Users\prime\anaconda3\envs\DL\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\prime\anaconda3\envs\DL\lib\site-packages\transformers\models\imagegpt\modeling_imagegpt.py:1143: FutureWarning: The `pixel_values` argument is deprecated and will be removed in a future version, use `input_ids` instead.
  warnings.warn(


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 201326592 bytes.